# Focusing Data figure
Temporarily called figure 3. Could change. We want to smooth and plot bars with confidence intervals for the data in xixi's new experiments


In [1]:
library(dplyr)
library(ggplot2)
library(cowplot)
library(magrittr)
library(tidyr)
library(broom)
library(splines)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘cowplot’

The following object is masked from ‘package:ggplot2’:

    ggsave


Attaching package: ‘tidyr’

The following object is masked from ‘package:magrittr’:

    extract



ERROR: Error in library(broom): there is no package called ‘broom’


In [2]:
focusing = read.csv("./rawdata/focusing2.csv")
focusing %<>% rename(Substrate = Experiment)

Warning message:
In file(file, "rt"): cannot open file './rawdata/focusing2.csv': No such file or directory

ERROR: Error in file(file, "rt"): cannot open the connection


ERROR: Error in eval(expr, envir, enclos): object 'focusing' not found


In [3]:
focusing %<>% gather(Experiment,Concentration,Control, Add_ATP,Wait)
focusing$channel[focusing$Distance <= 100] <- 'left'
focusing$channel[focusing$Distance > 100 & focusing$Distance <= 220] <- 'center'
focusing$channel[focusing$Distance > 220] <- 'right'

ERROR: Error in eval(expr, envir, enclos): object 'focusing' not found


ERROR: Error in focusing$channel[focusing$Distance <= 100] <- "left": object 'focusing' not found


ERROR: Error in focusing$channel[focusing$Distance > 100 & focusing$Distance <= : object 'focusing' not found


ERROR: Error in focusing$channel[focusing$Distance > 220] <- "right": object 'focusing' not found


In [4]:
m <- function(df,m) {
    temp <- df %>%
        bootstrap(m) %>%
#          do(augment(glm(data = ., Concentration ~ ns(Distance, df = 6)),.))
         do(augment(loess(data = ., Concentration ~ Distance, span =0.5),.))
    temp
}

In [5]:
focusing_bs <- focusing %>%
    group_by(Substrate, Experiment) %>%
    do(m(.,25))

ERROR: Error in eval(expr, envir, enclos): object 'focusing' not found


Now we need to normalize each bootstrap sample integral to 1. We do this by sorting the distance then integrating concnetration with distances.

In [6]:
focusing_bs$channel <- 0
focusing_bs$channel[focusing_bs$Distance <= 100] <- 'left'
focusing_bs$channel[focusing_bs$Distance > 100 & focusing_bs$Distance <= 220] <- 'center'
focusing_bs$channel[focusing_bs$Distance > 220] <- 'right'
focusing_bs$channel <- factor(focusing_bs$channel, levels = c ('left','center','right'))

ERROR: Error in focusing_bs$channel <- 0: object 'focusing_bs' not found


ERROR: Error in focusing_bs$channel[focusing_bs$Distance <= 100] <- "left": object 'focusing_bs' not found


ERROR: Error in focusing_bs$channel[focusing_bs$Distance > 100 & focusing_bs$Distance <= : object 'focusing_bs' not found


ERROR: Error in focusing_bs$channel[focusing_bs$Distance > 220] <- "right": object 'focusing_bs' not found


ERROR: Error in factor(focusing_bs$channel, levels = c("left", "center", "right")): object 'focusing_bs' not found


In [7]:
focusing_bs %<>% group_by(replicate,Substrate,Experiment,channel) %>% arrange(Distance) %>%
    mutate(concentration.channel.integral = t(diff(Distance)) %*% Concentration[1:(length(Concentration)-1)])
focusing_bs %<>% group_by(replicate,Substrate,Experiment,channel) %>% arrange(Distance) %>%
    mutate(.fitted.channel.integral = t(diff(Distance)) %*% .fitted[1:(length(.fitted)-1)])
focusing_bs %<>% ungroup() %>% group_by(Substrate,Experiment,replicate) %>% 
    mutate(concentration.integral = sum(concentration.channel.integral))
focusing_bs %<>% ungroup() %>% group_by(Substrate,Experiment,replicate) %>% 
    mutate(.fitted.integral = sum(.fitted.channel.integral))
# focusing_bs %<>% ungroup() %>% group_by(Experiment,replicate,channel,Substrate) %>% 
#     mutate(concentration.normalized = sum(Integral)/mean(normalized))
focusing_bs$concentration.normalized <- focusing_bs$Concentration / focusing_bs$concentration.integral
focusing_bs$.fitted.normalized <- as.vector(focusing_bs$.fitted) / as.vector(focusing_bs$.fitted.integral)
focusing_bs$.fitted.normalized.channel.integral <- as.vector(focusing_bs$.fitted.channel.integral) / as.vector(focusing_bs$.fitted.integral )
glimpse(focusing_bs)

ERROR: Error in eval(expr, envir, enclos): object 'focusing_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'focusing_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'focusing_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'focusing_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'focusing_bs' not found


ERROR: Error in as.vector(focusing_bs$.fitted): object 'focusing_bs' not found


ERROR: Error in as.vector(focusing_bs$.fitted.channel.integral): object 'focusing_bs' not found


ERROR: Error in nrow(tbl): object 'focusing_bs' not found


In [8]:
alpha <- 0.05
bs_recap <- focusing_bs %>%
    group_by(Experiment, Substrate, Distance,channel) %>% 
    summarise(lo = quantile(.fitted.normalized,alpha/2), 
              hi = quantile(.fitted.normalized,1-alpha/2), 
              smooth = mean(.fitted.normalized), 
              orig = mean(concentration.normalized))
focusing_bs %<>% group_by(Experiment, Substrate, Distance,channel) %>% 
    mutate(lo = quantile(.fitted.normalized,alpha/2), hi = quantile(.fitted.normalized,1-alpha/2))

ERROR: Error in eval(expr, envir, enclos): object 'focusing_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'focusing_bs' not found


In [9]:
bychannel_bs <- focusing_bs %>% group_by(replicate,Substrate,Experiment,channel) %>% arrange(Distance) %>%
    summarise(Integral = t(diff(Distance)) %*% Concentration[1:(length(Concentration)-1)])
bychannel_bs %<>% ungroup() %>% group_by(Substrate,Experiment,replicate) %>% mutate(normalized = sum(Integral))
bychannel_bs %<>% ungroup() %>% group_by(Experiment,replicate,channel,Substrate) %>% mutate(concentration = sum(Integral)/mean(normalized))
check <- bychannel_bs %>% ungroup() %>% group_by(Substrate,Experiment,replicate) %>% summarise(sum = sum(concentration))
head(check)

ERROR: Error in eval(expr, envir, enclos): object 'focusing_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'bychannel_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'bychannel_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'bychannel_bs' not found


ERROR: Error in head(check): object 'check' not found


In [10]:
bychannel_recap <- bs_recap %>% 
    group_by(channel,Substrate,Experiment) %>% 
    summarise(orig = sum(orig), smooth = sum(smooth), lo = sum(lo), hi = sum(hi))

ERROR: Error in eval(expr, envir, enclos): object 'bs_recap' not found


In [11]:
# glimpse(bychannel_recap)
write.csv(filter(bychannel_recap, Experiment !="Wait"),"./bychannel_recap.csv")

ERROR: Error in filter_(.data, .dots = lazyeval::lazy_dots(...)): object 'bychannel_recap' not found


In [12]:
options(repr.plot.width = 5, repr.plot.height = 2)
integrated <- ggplot(data = filter(bychannel_recap, Experiment !="Wait") , 
                     aes(x=channel, y= smooth,fill = Experiment)) + 
    geom_bar(stat = "identity",position = "dodge") +
    xlab("Channel") +
    ylab("Integrated N.F.I") +
    facet_grid(. ~ Substrate,
        labeller = labeller(Substrate =c(DG = 'D-Glucose',LG = 'L-Glucose'))) +
    theme_bw() +
    theme(panel.border = element_rect(size = .8, colour = "black"),
         axis.text.y = element_text(size=0),
         legend.title=element_blank(),
         strip.text = element_text(size = 9, vjust = -150),
         strip.background = element_rect(fill = NA, size = NA),
         panel.grid.major = element_line()) + 
    annotate("rect", xmin = 0.5, xmax = 1.5, ymin = 0, ymax = +.0055, alpha = .2) +
    annotate("rect", xmin = 2.5, xmax = 3.5, ymin = 0, ymax = +.0055, alpha = .2) + 
    geom_bar(stat = "identity",position = "dodge") +
    geom_errorbar(aes(ymin = lo,ymax = hi ),position = position_dodge(1), width = 0) +
scale_fill_brewer(palette = 'Set1',labels = c("Control","ATP in\ncentral channel"))

# data frame for annotation bar

ann.data <- data.frame(Substrate = c("DG","LG"), Experiment = c("Control","Add_ATP"), 
                      ann.x = c(0.75, 0.75), ann.xend = c(1.25,1.25), ann.y = c(0.0033, 0.0037),
                      ann.yend = c(0.0033,0.0037), ann.text = c("***"," n.s."), ann.textoffset = c (0.0001,0.0004))

ann.data <- ann.data[rep(seq_len(nrow(ann.data)), each = 3),]
ann.data$channel <- factor(c('left','center','right','left','center','right'), levels = c ('left','center','right'))
#order is DG l,c,r LG l,c,r
ann.data$x.offset <- c(0,1,2,0,1,2)
ann.data$y.offset <- c(0,0.002,0,0,0.001,0)


integrated <- integrated + 
    geom_segment(data = ann.data, size=.3,
                 aes(x = ann.x+x.offset, y=ann.y+y.offset, xend=ann.xend+x.offset, yend=ann.yend+y.offset)) +
    geom_segment(data = ann.data, size = .3,
                 aes(x = ann.x+x.offset,xend=ann.x+x.offset,y=ann.y-0.0002+y.offset,yend=ann.y+y.offset)) +
    geom_segment(data = ann.data, size =.3,
                 aes(x = ann.xend+x.offset,xend=ann.xend+x.offset,y=ann.y-0.0002+y.offset,yend=ann.y+y.offset)) +
    geom_text(data = ann.data, size= 3.5,
              aes(x = ann.x+.25+x.offset, y = ann.y+ann.textoffset+y.offset,label = ann.text))
                
print(integrated)

ERROR: Error in filter_(.data, .dots = lazyeval::lazy_dots(...)): object 'bychannel_recap' not found


ERROR: Error in eval(expr, envir, enclos): object 'integrated' not found


ERROR: Error in print(integrated): object 'integrated' not found


In [13]:
options(repr.plot.width = 5, repr.plot.height = 2)

raw <- ggplot(data = filter(bs_recap, Experiment != "Wait"),aes(x = Distance-160,color = Experiment)) + 
    geom_point(aes(y = orig), size = 0.1) +
    geom_line(aes(y = smooth)) +
    ylab("N.F.I.") +
    xlab(expression(paste("Distance from center (", mu, "m)", sep = ""))) +
    facet_grid(. ~ Substrate,
               labeller = labeller(Substrate =c(DG = 'D-Glucose',LG = 'L-Glucose'))) +
    theme_bw() +     
    theme(panel.border = element_rect(size = 0.8, colour = "black"),
        axis.text.y = element_text(size=0),
        legend.title=element_blank(),
         strip.text = element_text(size = 9, vjust = -150),
         strip.background = element_rect(fill = NA, size = NA)) +
    annotate("rect", xmin = -160, xmax = -60, ymin = 0.0000, ymax = +.000091, alpha = .2) +
    annotate("rect", xmin = 60, xmax = 160, ymin = 0.00000, ymax = +.000091, alpha = .2) +
scale_color_brewer(palette = 'Set1',labels = c("Control","ATP in\ncentral channel"))

    
print(raw)

ERROR: Error in filter_(.data, .dots = lazyeval::lazy_dots(...)): object 'bs_recap' not found


function (length = 0L) 
.Internal(vector("raw", length))
<bytecode: 0x2071150>
<environment: namespace:base>


In [14]:
options(repr.plot.width = 5, repr.plot.height = 4)

plot_grid(raw,integrated, labels = c("A","B"),ncol = 1)

ERROR: Error in plot_grid(raw, integrated, labels = c("A", "B"), ncol = 1): object 'integrated' not found


In [15]:
ggsave("./img/FocusingExperimentPlot.png",width = 5, height = 4)

ERROR: Error in UseMethod("grid.draw"): no applicable method for 'grid.draw' applied to an object of class "NULL"


In [16]:
subset <- filter(bychannel_bs,Substrate == "LG", channel == "left")
pairwise.t.test(x = subset$concentration, g =subset$Experiment)

ERROR: Error in filter_(.data, .dots = lazyeval::lazy_dots(...)): object 'bychannel_bs' not found


ERROR: Error in subset$Experiment: object of type 'closure' is not subsettable


In [17]:
subset <- filter(bychannel_bs,Substrate == "DG", channel == "left")
pairwise.t.test(x = subset$concentration, g =subset$Experiment)

ERROR: Error in filter_(.data, .dots = lazyeval::lazy_dots(...)): object 'bychannel_bs' not found


ERROR: Error in subset$Experiment: object of type 'closure' is not subsettable


In [18]:
my.anova <- aov(formula = concentration.integral*10000 ~ Experiment, data = subset)

ERROR: Error in terms.formula(formula, "Error", data = data): 'data' argument is of the wrong type


In [19]:
summary(my.anova)

ERROR: Error in summary(my.anova): object 'my.anova' not found


In [20]:
pairwise.t.test(x = subset$concentration.integral, g =subset$Experiment)

ERROR: Error in subset$Experiment: object of type 'closure' is not subsettable


In [21]:
ggplot(data = subset, aes(x = Experiment, y=concentration)) + geom_boxplot()

ERROR: Error in if (is.waive(data) || empty(data)) return(cbind(data, PANEL = integer(0))): missing value where TRUE/FALSE needed


ERROR: Error in file(con, "rb"): cannot open the connection


ERROR: Error in file(con, "rb"): cannot open the connection


plot without title

In [22]:
subset <- filter(bychannel_bs,Substrate == "LG", channel == "left")
control_LG <- filter(subset, Experiment == "Control")$concentration
atp_LG <- filter(subset, Experiment == "Add_ATP")$concentration

ERROR: Error in filter_(.data, .dots = lazyeval::lazy_dots(...)): object 'bychannel_bs' not found


ERROR: Error in UseMethod("filter_"): no applicable method for 'filter_' applied to an object of class "function"


ERROR: Error in UseMethod("filter_"): no applicable method for 'filter_' applied to an object of class "function"


In [23]:
mean(control_LG)

ERROR: Error in mean(control_LG): object 'control_LG' not found


In [24]:
mean(atp_LG)

ERROR: Error in mean(atp_LG): object 'atp_LG' not found


In [25]:
sd_ctrl_LG <- sd(control_LG)
sd_atp_LG <- sd(atp_LG)

ERROR: Error in is.data.frame(x): object 'control_LG' not found


ERROR: Error in is.data.frame(x): object 'atp_LG' not found


In [26]:
sd_atp_LG

ERROR: Error in eval(expr, envir, enclos): object 'sd_atp_LG' not found


In [27]:
sd_ctrl_LG

ERROR: Error in eval(expr, envir, enclos): object 'sd_ctrl_LG' not found


In [28]:
t <- (mean(control_LG)-mean(atp_LG)) / sqrt(var(control_LG)/25+var(atp_LG)/25)

ERROR: Error in mean(control_LG): object 'control_LG' not found


In [29]:
t

function (x) 
UseMethod("t")
<bytecode: 0x3c804b8>
<environment: namespace:base>

In [30]:
t.test(control_LG,atp_LG)

ERROR: Error in t.test(control_LG, atp_LG): object 'control_LG' not found


# Figure 3, August 15 2016
Redoing figure 3 with xixi's new data for focusing for Mannose, and using AMPCPP. Also, just received the triplicate data for the glucose experiment, so should account for this as well. Since the format is different than before i am going to copy the data anaysis part from RoleOfCatalysisPlot-v4.

In [31]:
fig3<- read.csv("../newdata/alldata_081516.csv")

In [32]:
n<-dim(fig3)[1]
fig3<-fig3[1:(n-1),]

In [33]:
tail(fig3)

,distance,dg,dg.ctrl,amppcp,amppcp.ctrl,mannose.ctrl,mannose,lg.ctrl,lg
161,302.1419,33.7738,43.3214,114.667,129.333,161.333,137.5,37,44
162,304.0303,31.6875,41.6875,108.667,131.667,160.5,136,39.6667,40.6667
163,305.9187,31,37.9286,108.333,131,153.667,135.333,40,41.3333
164,307.8071,30.2351,37,107,131.333,142,137.5,39,41
165,309.6955,29.2381,37,111.333,130,138,128.667,38.6667,40.3333
166,311.5839,26.7232,38.5179,108,133,134.5,132.167,36.3333,37


In [34]:
fig3 <- gather(fig3,"Experiment","Concentration",dg:lg)

In [35]:
head(fig3)

,distance,Experiment,Concentration
1,0,dg,31.25
2,1.8884,dg,32.247
3,3.7768,dg,33
4,5.6652,dg,33.4821
5,7.5535,dg,35.2381
6,9.4419,dg,36


In [36]:
 normalit<-function(m){
   (m - min(m))/sum(m-min(m))
 }

In [37]:
library(stringr)

In [38]:
fig3 %<>% group_by(Experiment) %>% mutate(Concentration.norm = normalit(Concentration))

In [39]:
fig3$Substrate = NA
fig3$Substrate <- str_extract(fig3$Experiment, "[a-z]+")
fig3$Experiment <- str_match(fig3$Experiment, "[.]([a-z]+)")[,2]
fig3$Experiment[is.na(fig3$Experiment)] = "substrate"

In [40]:
fig3[200:210,]

,distance,Experiment,Concentration,Concentration.norm,Substrate
1,62.3168,ctrl,46.5446,0.005145322,dg
2,64.2052,ctrl,44,0.00424514,dg
3,66.0936,ctrl,44.2917,0.004348332,dg
4,67.9819,ctrl,46.1429,0.005003216,dg
5,69.8703,ctrl,47.4196,0.005454864,dg
6,71.7587,ctrl,49.7262,0.00627085,dg
7,73.6471,ctrl,47.7321,0.005565414,dg
8,75.5355,ctrl,46.131,0.004999006,dg
9,77.4239,ctrl,46.872,0.005261144,dg
10,79.3123,ctrl,46,0.004952664,dg


In [41]:
fig3$channel <- NA
fig3$channel[fig3$distance <= 100] <- 'left'
fig3$channel[fig3$distance > 100 & fig3$Distance <= 220] <- 'center'
fig3$channel[fig3$distance > 220] <- 'right'

In [42]:
head(fig3)

,distance,Experiment,Concentration,Concentration.norm,Substrate,channel
1,0,substrate,31.25,0.001244312,dg,left
2,1.8884,substrate,32.247,0.001518365,dg,left
3,3.7768,substrate,33,0.001725347,dg,left
4,5.6652,substrate,33.4821,0.001857865,dg,left
5,7.5535,substrate,35.2381,0.002340549,dg,left
6,9.4419,substrate,36,0.002549977,dg,left


In [43]:
fig3$Substrate <- factor(fig3$Substrate, levels = c ('dg','mannose','lg','amppcp'))

In [44]:
m <- function(df,m) {
    temp <- df %>%
        bootstrap(m) %>%
#          do(augment(glm(data = ., Concentration ~ ns(Distance, df = 6)),.))
         do(augment(loess(data = ., Concentration ~ distance, span =0.5),.))
    temp
}

In [45]:
fig3_bs <- fig3 %>%
    group_by(Substrate,Experiment) %>%
    do(m(.,200))

ERROR: Error in function_list[[i]](value): could not find function "bootstrap"


In [46]:
fig3_bs$channel <- 0
fig3_bs$channel[fig3_bs$distance <= 100] <- 'left'
fig3_bs$channel[fig3_bs$distance > 100 & fig3_bs$distance <= 220] <- 'center'
fig3_bs$channel[fig3_bs$distance > 220] <- 'right'
fig3_bs$channel <- factor(fig3_bs$channel, levels = c ('left','center','right'))

ERROR: Error in fig3_bs$channel <- 0: object 'fig3_bs' not found


ERROR: Error in fig3_bs$channel[fig3_bs$distance <= 100] <- "left": object 'fig3_bs' not found


ERROR: Error in fig3_bs$channel[fig3_bs$distance > 100 & fig3_bs$distance <= : object 'fig3_bs' not found


ERROR: Error in fig3_bs$channel[fig3_bs$distance > 220] <- "right": object 'fig3_bs' not found


ERROR: Error in factor(fig3_bs$channel, levels = c("left", "center", "right")): object 'fig3_bs' not found


In [47]:
fig3_bs %<>% group_by(replicate,Substrate,Experiment,channel) %>% arrange(distance) %>%
    mutate(concentration.channel.integral = t(diff(distance)) %*% Concentration[1:(length(Concentration)-1)])
fig3_bs %<>% group_by(replicate,Substrate,Experiment,channel) %>% arrange(distance) %>%
    mutate(.fitted.channel.integral = t(diff(distance)) %*% .fitted[1:(length(.fitted)-1)])
fig3_bs %<>% ungroup() %>% group_by(Substrate,Experiment,replicate) %>% 
    mutate(concentration.integral = sum(concentration.channel.integral))
fig3_bs %<>% ungroup() %>% group_by(Substrate,Experiment,replicate) %>% 
    mutate(.fitted.integral = sum(.fitted.channel.integral))
# fig3_bs %<>% ungroup() %>% group_by(Experiment,replicate,channel,Substrate) %>% 
#     mutate(concentration.normalized = sum(Integral)/mean(normalized))
fig3_bs$concentration.normalized <- fig3_bs$Concentration / fig3_bs$concentration.integral
fig3_bs$.fitted.normalized <- as.vector(fig3_bs$.fitted) / as.vector(fig3_bs$.fitted.integral)
fig3_bs$.fitted.normalized.channel.integral <- as.vector(fig3_bs$.fitted.channel.integral) / as.vector(fig3_bs$.fitted.integral )
glimpse(fig3_bs)

ERROR: Error in eval(expr, envir, enclos): object 'fig3_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'fig3_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'fig3_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'fig3_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'fig3_bs' not found


ERROR: Error in as.vector(fig3_bs$.fitted): object 'fig3_bs' not found


ERROR: Error in as.vector(fig3_bs$.fitted.channel.integral): object 'fig3_bs' not found


ERROR: Error in nrow(tbl): object 'fig3_bs' not found


In [48]:
alpha <- 0.05
fig3_recap <- fig3_bs %>%
    group_by(Experiment, Substrate, distance,channel) %>% 
    summarise(lo = quantile(.fitted.normalized,alpha/2), 
              hi = quantile(.fitted.normalized,1-alpha/2), 
              smooth = mean(.fitted.normalized), 
              orig = mean(concentration.normalized))
fig3_bs %<>% group_by(Experiment, Substrate, distance,channel) %>% 
    mutate(lo = quantile(.fitted.normalized,alpha/2), hi = quantile(.fitted.normalized,1-alpha/2))

ERROR: Error in eval(expr, envir, enclos): object 'fig3_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'fig3_bs' not found


In [49]:
bychannel_bs <- fig3_bs %>% group_by(replicate,Substrate,Experiment,channel) %>% arrange(distance) %>%
    summarise(Integral = t(diff(distance)) %*% Concentration[1:(length(Concentration)-1)])
bychannel_bs %<>% ungroup() %>% group_by(Substrate,Experiment,replicate) %>% mutate(normalized = sum(Integral))
bychannel_bs %<>% ungroup() %>% group_by(Experiment,replicate,channel,Substrate) %>% mutate(concentration = sum(Integral)/mean(normalized))
check <- bychannel_bs %>% ungroup() %>% group_by(Substrate,Experiment,replicate) %>% summarise(sum = sum(concentration))
head(check)

ERROR: Error in eval(expr, envir, enclos): object 'fig3_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'bychannel_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'bychannel_bs' not found


ERROR: Error in eval(expr, envir, enclos): object 'bychannel_bs' not found


ERROR: Error in head(check): object 'check' not found


In [50]:
bychannel_recap <- fig3_recap %>% 
    group_by(channel,Substrate,Experiment) %>% 
    summarise(orig = sum(orig), smooth = sum(smooth), lo = sum(lo), hi = sum(hi))

ERROR: Error in eval(expr, envir, enclos): object 'fig3_recap' not found


In [51]:
options(repr.plot.width = 7.5, repr.plot.height = 2)
integrated <- ggplot(data = bychannel_recap, 
                     aes(x=channel, y= smooth,fill = Experiment)) + 
    geom_bar(stat = "identity",position = "dodge") +
    xlab("Channel") +
    ylab("Integrated N.F.I") +
    facet_grid(. ~ Substrate,
       labeller = labeller(Substrate =c(dg = 'D-Glucose/ATP',lg = 'L-Glucose/ATP', mannose = 'Mannose/ATP',
                                       amppcp = "D-Glucose/AMP-PCP"))) +
    theme_bw() +
    theme(panel.border = element_rect(size = .8, colour = "black"),
         axis.text.y = element_text(size=0),
         legend.title=element_blank(),
         strip.text = element_text(size = 9, vjust = -150),
         strip.background = element_rect(fill = NA, size = NA),
         panel.grid.major = element_line()) + 
    annotate("rect", xmin = 0.5, xmax = 1.5, ymin = 0, ymax = +.0056, alpha = .2) +
    annotate("rect", xmin = 2.5, xmax = 3.5, ymin = 0, ymax = +.0056, alpha = .2) + 
    geom_bar(stat = "identity",position = "dodge") +
    geom_errorbar(aes(ymin = lo,ymax = hi ),position = position_dodge(1), width = 0) +
scale_fill_brewer(palette = 'Set1',labels = c("Control","Substrate in\ncentral channel"))

# data frame for annotation bar

ann.data <- data.frame(Substrate = c("dg","mannose","lg","amppcp"), Experiment = c("substrate","ctrl"), 
                      ann.x = c(0.75, 0.75), ann.xend = c(1.25,1.25), ann.y = c(0.0033, 0.0037),
                      ann.yend = c(0.0033,0.0037), ann.text = c("***","***"," n.s."," n.s."), 
                       ann.textoffset = c (0.0001,0.0001,0.0004,0.0004))

ann.data <- ann.data[rep(seq_len(nrow(ann.data)), each = 1),]
#ann.data$channel <- factor(c('left','center','right','left','center','right'), levels = c ('left','center','right'))
ann.data$channel <- factor(c('left'))
#order is DG l,c,r LG l,c,r
ann.data$x.offset <- c(1,1,1,1)
ann.data$y.offset <- c(0.0021,0.0012,0.0012,0.001)


integrated <- integrated + 
    geom_segment(data = ann.data, size=.3,
                 aes(x = ann.x+x.offset, y=ann.y+y.offset, xend=ann.xend+x.offset, yend=ann.yend+y.offset)) +
    geom_segment(data = ann.data, size = .3,
                 aes(x = ann.x+x.offset,xend=ann.x+x.offset,y=ann.y-0.0002+y.offset,yend=ann.y+y.offset)) +
    geom_segment(data = ann.data, size =.3,
                 aes(x = ann.xend+x.offset,xend=ann.xend+x.offset,y=ann.y-0.0002+y.offset,yend=ann.y+y.offset)) +
    geom_text(data = ann.data, size= 3.5,
              aes(x = ann.x+.25+x.offset, y = ann.y+ann.textoffset+y.offset,label = ann.text))
                
print(integrated)

ERROR: Error in ggplot(data = bychannel_recap, aes(x = channel, y = smooth, fill = Experiment)): object 'bychannel_recap' not found


ERROR: Error in eval(expr, envir, enclos): object 'integrated' not found


ERROR: Error in print(integrated): object 'integrated' not found


In [52]:
ann.data

,Substrate,Experiment,ann.x,ann.xend,ann.y,ann.yend,ann.text,ann.textoffset,channel,x.offset,y.offset
1,dg,substrate,0.75,1.25,0.0033,0.0033,***,1e-04,left,1,0.0021
2,mannose,ctrl,0.75,1.25,0.0037,0.0037,***,1e-04,left,1,0.0012
3,lg,substrate,0.75,1.25,0.0033,0.0033,n.s.,4e-04,left,1,0.0012
4,amppcp,ctrl,0.75,1.25,0.0037,0.0037,n.s.,4e-04,left,1,0.001


In [53]:
options(repr.plot.width = 7.5, repr.plot.height = 2)

raw <- ggplot(data = filter(fig3_recap),aes(x = distance-160,color = Experiment)) + 
    geom_point(aes(y = orig), size = 0.1) +
    geom_line(aes(y = smooth)) +
    ylab("N.F.I.") +
    xlab(expression(paste("Distance from center (", mu, "m)", sep = ""))) +
    facet_grid(. ~ Substrate,
               labeller = labeller(Substrate =c(dg = 'D-Glucose/ATP',lg = 'L-Glucose/ATP', mannose = 'Mannose/ATP',
                                       amppcp = "D-Glucose/AMP-PCP"))) +
    theme_bw() +     
    theme(panel.border = element_rect(size = 0.8, colour = "black"),
        axis.text.y = element_text(size=0),
        legend.title=element_blank(),
         strip.text = element_text(size = 9, vjust = -150),
         strip.background = element_rect(fill = NA, size = NA)) +
    annotate("rect", xmin = -160, xmax = -60, ymin = 0.00002, ymax = +.000092, alpha = .2) +
    annotate("rect", xmin = 60, xmax = 160, ymin = 0.00002, ymax = +.000092, alpha = .2) +
scale_color_brewer(palette = 'Set1',labels = c("Control","Substrate in\ncentral channel"))

    
print(raw)

ERROR: Error in filter_(.data, .dots = lazyeval::lazy_dots(...)): object 'fig3_recap' not found


function (length = 0L) 
.Internal(vector("raw", length))
<bytecode: 0x2071150>
<environment: namespace:base>


In [54]:
options(repr.plot.width = 7.6, repr.plot.height = 4)

plot_grid(raw,integrated, labels = c("A","B"),ncol = 1)

ERROR: Error in plot_grid(raw, integrated, labels = c("A", "B"), ncol = 1): object 'integrated' not found


In [55]:
ggsave("../img/FocusingExperimentPlot_v3.png",width = 7.6, height = 4)

ERROR: Error in if (is.waive(data) || empty(data)) return(cbind(data, PANEL = integer(0))): missing value where TRUE/FALSE needed


In [56]:
subset <- filter(bychannel_bs,Substrate == "dg", channel == "center")
control_LG <- filter(subset, Experiment == "ctrl")$concentration
atp_LG <- filter(subset, Experiment == "substrate")$concentration

ERROR: Error in filter_(.data, .dots = lazyeval::lazy_dots(...)): object 'bychannel_bs' not found


ERROR: Error in UseMethod("filter_"): no applicable method for 'filter_' applied to an object of class "function"


ERROR: Error in UseMethod("filter_"): no applicable method for 'filter_' applied to an object of class "function"


In [57]:
t.test(atp_LG,control_LG,alternative = 'greater')

ERROR: Error in t.test(atp_LG, control_LG, alternative = "greater"): object 'atp_LG' not found
